In [42]:
from rosenbrock import rosenbrock
from kenny import kenny
import numpy.matlib as mat
import numpy as np
# DIVIDE Zero oli!!!
# User are in order as follows: Nr of population(N_p), Nr. of iterations(T), lower bound(lb), upper bounds(ub),
#optimization problem(prob), number of search variables(N_variables), Scaling factor minimum bound(F_min), 
# maximum bound(F_max), adaptive crossover minimum (P_c_min), maximum(P_c_max)


#Problem settings

#Paramaters for Differential Evolution

N_p = 4 #Number of population
T = 200 #number of iterations

lb =-10 #searchspace lower bound
ub =10  #searchspace upper bound

prob = rosenbrock #the fitness function
N_variables=2 #Number of search variables from fitness function

#Paramaters for adaptive  Scaling FACTOR F (between 0-2)
F_min=0.8 #minimum of Scaling Factor
F_max=1   #maximum of Scaling Factor

#Paramaters for adaptive crossover probability
P_c_min=0.5
P_c_max=1

#THe function aadioptimiz is the function itself it takes arguments 
#in same order as the problem settings.
best_of_f, globopt = aadioptimize(N_p,T,lb,ub,prob,F_min,F_max,P_c_min,P_c_max)
#Found optimization location
found_loc =globopt
known_loc=[1,1]
#distance from known location

print(f'Distance between found and known global optimum loc: {distance(known_loc,found_loc,N_variables)}')
print(f'Global optimum location at: {found_loc}')
print(f'Best fitness function value found was:{best_of_f}')

Distance between found and known global optimum loc: 1.6579547780638164
Global optimum location at: [-0.47313212  0.23927941]
Best fitness function value found was:[2.19391257]


In [41]:

        

        def initDE(N_p,lb,ub,prob):
            lb = np.full(N_p,lb)
            
            ub = np.full(N_p,ub)
            
            f = np.zeros((N_p,1)) #empty vector for fitness function
            
            fu = np.zeros((N_p,1))#newly created trial vector

            D = len(lb) # Determining amount of decision variables
    
            U = np.zeros((N_p,D)) #Matrix for storing trial solutions 
        
            #Initial random population !!!!!MIND THE "LEN", I DONT KNOW if that works"
            P = mat.repmat(lb,N_p,1)+mat.repmat((ub-lb),N_p,1)*np.random.rand(len(ub-lb),N_p)
        
            for p in np.arange(N_p):
                f[p]=prob(P[p,])
            
            return lb,ub,f,fu,D,U,P
        
        
        
    
         #This function starts the mutation process and generates a donorvector
        def mutation1(i,N_p,t,T,P):
            #Adaptive scaling factor
            if N_p > 3:
                F=F_min*2**np.exp(1-(T/(T+1-t)))
            else:
                F = 0.5
            #candidates are assigned without the i-th element
            candidates= np.delete(np.arange(N_p), np.where(np.arange(N_p)==i))
            #3 target vectors are picked out randomly for the donorvector generator
            cand_rand=np.random.choice(candidates,3,replace= False)
            X1=P[cand_rand[0],]
            X2=P[cand_rand[1],]
            X3=P[cand_rand[2],]
       
            #Donorvctor generator
            V= X1 + F*(X2-X3)
            return V
        
        
        #Donor vector with minimum of 6 N_p, does not work really well compared 
        #to mutation1

        #This function starts the mutation process and generates a donorvector
        def mutation2(i,N_p,t,T,P):
            #Adaptive scaling factor
            F=F_min*2**np.exp(1-(T/(T+1-t)))
            #candidates are assigned without the i-th element
            candidates= np.delete(np.arange(N_p), np.where(np.arange(N_p)==i))
            cand_rand=np.random.choice(candidates,5,replace= False)
            X1=P[cand_rand[0],]
            X2=P[cand_rand[1],]
            X3=P[cand_rand[2],]
            X4=P[cand_rand[3],]
            X5=P[cand_rand[4],]
            #Donorvector generator
            V = X1 + F*(X2-X3)+ F*(X4 -X5)
    
            return V
        
        
        
    
        #this function evaluates donor vector and uses parts of it which fit better
        def crossover(f,P_c_min,P_c_max,i,D,V,P,U):
            #ADAPTIVE Crossover
            if f[i] < np.mean(f):
                P_c = P_c_min + (P_c_max-P_c_min)*((f[i]-np.mean(f))/(np.max(f)-np.mean(f)))
            else:
                P_c = P_c_min
        
            delta = np.random.randint(0,D-1) 
            for j in np.arange(D):
                if np.random.uniform(0,1) <= P_c or delta == j:
                    U[i,j] = V[j]
                else:
                    U[i,j]=P[i,j]
        
            return U
        
        
        
        
        
        #this function bounds the vector and replaces the old target vector with new if better
        def boundgreed(j,U,P,f,fu,ub,lb,prob):
            
            U[j]=np.minimum(U[j], ub)
            U[j]=np.maximum(U[j], lb)
    
            fu[j]=prob(U[j])

            if fu[j] < f[j]:
                P[j]= U[j]
                f[j]=fu[j]
            return fu,f,P
        
        #distance from known location
        def distance(known_loc,found_loc,N_variables,):
            known = known_loc
            opt = found_loc
            undersqrt=np.zeros(N_variables)
            for i in (np.arange(N_variables)):
                undersqrt[i]  =(known_loc[i]-found_loc[i])**2
                dist = np.sqrt(sum(undersqrt))
        
            return dist

        
        
        def aadioptimize(N_p,T,lb,ub,prob,F_min,F_max,P_c_min,P_c_max):
            lb,ub,f,fu,D,U,P = initDE(N_p,lb,ub,prob)
            if N_p < 4:
                raise Exception("Sorry, there must be atleast a population of 4. Reccomended 20")
            
            for t in np.arange(T):
                for i in np.arange(N_p):
        
                    V = mutation1(i,N_p,t,T,P)
        
                    U=crossover(f,P_c_min,P_c_max,i,D,V,P,U)
    
                for j in np.arange(N_p):    
        
                    fu,f,P = boundgreed(j,U,P,f,fu,ub,lb,prob)
        
            best_of_f= min(f)
            globopt = P[f.argmin()]
            return best_of_f, globopt[:N_variables]
        
        
 

    

In [ ]:
#Donor vector with minimum of 6 N_p

        #This function starts the mutation process and generates a donorvector
def mutation2(i,N_p,t,T,P):
    #Adaptive scaling factor
    F=F_min*2**np.exp(1-(T/(T+1-t)))
    #candidates are assigned without the i-th element
    candidates= np.delete(np.arange(N_p), np.where(np.arange(N_p)==i))
    cand_rand=np.random.choice(candidates,5,replace= False)
    X1=P[cand_rand[0],]
    X2=P[cand_rand[1],]
    X3=P[cand_rand[2],]
    X4=P[cand_rand[3],]
    X5=P[cand_rand[4],]
               
    #Donorvector generator with at least 6 N_p
    V = X1 + F(X2-X3)+ F(X4 -X5)
    
    return V